In [28]:
#####RESCAL#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import math
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
import seaborn as sns

#np.random.seed(98537)

In [29]:
####データの発生####
##データの設定
k = 10
d = 5000
m = 12
no = np.arange(d)

In [30]:
##ノードを生成
#セグメントをリンク確率を生成
g = 15
Z = np.random.multinomial(1, np.random.dirichlet(np.repeat(2.0, g), 1).reshape(-1), d)
z_vec = np.dot(Z, np.arange(g))
beta = np.random.beta(0.75, 15.0, g*g).reshape(g, g)
beta = np.triu(beta, k=1) + np.triu(beta).T

#ノードの全パターンを作成
target = np.triu(np.full((d, d), 1), k=1)
D1 = np.triu(np.full((d, d), np.arange(d)).T, k=1)
D2 = np.triu(np.full((d, d), np.arange(d)), k=1)
d1 = D1[target==1]; d2 = D2[target==1]
del target; D1, D2

#ユーザーごとにノードを生成
y = np.random.binomial(1, np.sum(beta[z_vec[d1], ] * Z[d2, ], axis=1), d1.shape[0])
index_y = np.where(y==1)[0]
d_id1 = d1[index_y]; d_id2 = d2[index_y]

#インデックスを作成
target = [i for i in range(d)]
d_list = [i for i in range(d)]
d_list1 = [i for i in range(d)]
d_list2 = [i for i in range(d)]
pt = np.repeat(0, d)
for i in range(d):
    d_list1[i] = np.array(np.where(d_id1==i)[0], dtype="int32")
    d_list2[i] = np.array(np.where(d_id2==i)[0], dtype="int32")
    d_list[i] = np.append(d_list1[i], d_list2[i])
    target[i] = np.append(np.repeat(0, d_list1[i].shape[0]), np.repeat(1, d_list2[i].shape[0]))
    pt[i] = target[i].shape[0]
N = np.sum(pt)
hhpt = d_id1.shape[0]

In [31]:
##説明変数の生成
#素性ベクトルを生成
k1 = 4; k2 = 5; k3 = 4
x1 = np.array(np.random.random(hhpt*k1)).reshape((hhpt, k1))
x2 = np.zeros((hhpt, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    x2[:, j] = np.random.binomial(1, prob, hhpt)
x3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), hhpt)
x3 = np.delete(x3, np.argmin(np.sum(x3, axis=0)), axis=1)   #冗長な変数の削除
x = np.concatenate((np.repeat(1, hhpt)[:, np.newaxis], x1, x2, x3), axis=1)
x_col = x.shape[1]

In [32]:
####応答変数を生成####
rp = 0
while True:
    rp = rp + 1
    print(rp)

    #モデルパラメータを生成
    beta = np.append(np.random.uniform(-1.25, -0.5, m), np.random.normal(0, 0.3, (x_col-1)*m)).reshape(x_col, m)
    Cov = np.diag(np.repeat(0.5, k))
    theta = np.random.multivariate_normal(np.repeat(0, k), Cov, d)
    omega = np.zeros((k, k, m))
    for j in range(m):
        omega[:, :, j] = np.random.normal(0, 0.3, k*k).reshape(k, k)
    betat = beta.copy(); thetat = theta.copy(); omegat = omega.copy()
        
    ##応答変数を生成
    #期待値を設定
    beta_mu = np.dot(x, beta)
    mu = np.zeros((hhpt, m))
    for j in range(m):
        mu[:, j] = beta_mu[:, j] + np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))

    #正規分布からバイナリー変数を生成
    U = np.random.normal(mu, 1)
    y = np.array(U > 0, dtype="int")
    UT = U.copy()
    
    #break条件
    if (np.min(np.mean(y, axis=0)) > 0.1) & (np.max(np.mean(y, axis=0)) < 0.5):
        break

1


In [33]:
#生成したデータを確認
print(np.round(np.mean(y, axis=0), 3))
print(np.sum(y, axis=0))

[0.303 0.462 0.223 0.286 0.355 0.442 0.409 0.263 0.182 0.368 0.264 0.379]
[156788 239255 115469 148262 183814 228794 211701 136168  94154 190639
 136598 196069]


In [34]:
####RESCALのパラメータを推定####
##パラメータ推定のための関数
#切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n, k):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    if k==1:
        res = norm.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)
    else:
        res = norm.ppf(np.random.uniform(0, 1, n*k).reshape(n, k)*(FB-FA)+FA, mu, sigma)
    return res

#対数尤度関数の和を計算する関数
def loglike(omega, theta, theta_vec1, theta_vec2, Sigma, er, d_id1, d_id2, hhpt, k, m):
    #期待値を設定
    ARA = np.zeros((hhpt, m))
    for j in range(m):
        ARA[:, j] = np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta_vec2, np.repeat(1, k))    
    mu = beta_mu + ARA

    #対数尤度の和
    LLi = scipy.stats.norm.logpdf(er, mu, Sigma)
    LL = np.sum(LLi, axis=0)
    return LL

In [35]:
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 2000
keep = 2  
iter = 0
burnin = 250/keep
disp = 10
e = 0.1
L = 3

In [36]:
#正則パラメータを設定
Cov = np.diag(np.repeat(10, k))
inv_Cov = np.linalg.inv(Cov)

In [37]:
##データの設定
#切断領域を定義
a = (1-y)*(-100) + y*0
b = y*100 + (1-y)*0

#回帰モデルの定数を設定
xx = np.dot(x.T, x)
inv_xx = np.linalg.inv(xx)

#インデックスを設定
pt_m = pt*m
allocation_list1 = [i for i in range(d)]
for i in range(d):
    allocation_list1[i] = np.arange(pt[i]*m).reshape(pt[i], m, order="F")
allocation_list2 = np.arange(k*k).reshape(k, k, order="F")

In [52]:
##パラメータの真値
#モデルパラメータの真値
Sigma = 1
beta = betat.copy()
theta = thetat.copy()
omega = omegat.copy()

#期待値の真値
beta_mu = np.dot(x, beta)
mu = np.zeros((hhpt, m))
for j in range(m):
    mu[:, j] = beta_mu[:, j] + np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))

In [24]:
##パラメータの初期値
#モデルパラメータの初期値
Sigma = 1
beta = np.append(np.random.uniform(-1.0, -0.5, m), np.random.normal(0, 0.1, (x_col-1)*m)).reshape(x_col, m)
theta = np.random.multivariate_normal(np.repeat(0, k), np.diag(np.repeat(0.01, k)), d)
omega = np.random.normal(0, 0.1, k*k*m).reshape(k, k, m)

#期待値の初期値
beta_mu = np.dot(x, beta)
mu = np.zeros((hhpt, m))
for j in range(m):
    mu[:, j] = beta_mu[:, j] + np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))

In [39]:
##対数尤度の基準値
#1パラメータでの対数尤度
LLst1 = np.sum(y*np.full((hhpt, m), np.log(np.mean(y, axis=0))))
LLst0 = np.sum((1-y)*np.full((hhpt, m), np.log(np.mean(1-y, axis=0))))
LLst = LLst1 + LLst0
print(LLst)

#真値での対数尤度
mu = np.zeros((hhpt, m))
for j in range(m):
    mu[:, j] = np.dot(x, betat[:, j]) + np.dot(np.dot(thetat, omegat[:, :, j])[d_id1, ] * thetat[d_id2, ], np.repeat(1, k))
Prob = norm.cdf(mu, 0, Sigma)
Prob[Prob==1.0] = 0.99999999999; Prob[Prob==0] = np.power(10.0, -100)
LLbest = np.sum(y*np.log(Prob)) + np.sum((1-y)*np.log(1-Prob))
print(LLbest)

-3828369.186995712
-2189804.6230906798


In [63]:
##切断正規分布から潜在変数をサンプリング
#期待値の設定
theta_vec1 = theta[d_id1, ]; theta_vec2 = theta[d_id2, ]
beta_mu = np.dot(x, beta)
ARA = np.zeros((hhpt, m))
AR = [j for j in range(m)]; RA = [j for j in range(m)]
for j in range(m):
    AR[j] = np.dot(theta, omega[:, :, j])[d_id1, ]; RA[j] = np.dot(theta, omega[:, :, j].T)[d_id2, ]
    ARA[:, j] = np.dot(AR[j] * theta_vec2, np.repeat(1, k))
mu = beta_mu + ARA

#切断領域に応じて潜在変数をサンプリング
U = rtnorm(mu, Sigma, a, b, hhpt, m)

In [ ]:
##モデルパラメータを更新
#素性ベクトルの回帰ベクトルを更新
er = U - ARA   #モデル誤差
xy = np.dot(x.T, er)
beta = np.dot(inv_xx, xy)

In [96]:
##ノードごとに特徴ベクトルを更新
#モデル誤差を設定
er = U - beta_mu
er_T = er.T

for i in range(d):
    #データの定義
    v = np.zeros((pt_m[i], k))
    for j in range(m):
        index = allocation_list1[i]
        v[index[:, j], ] = np.vstack((RA[j][d_list1[i], ], AR[j][d_list2[i], ]))

    #特徴ベクトルを更新
    vy = np.dot(v.T, er_T[:, d_list[i]].reshape(-1))
    vv = np.dot(v.T, v) + inv_Cov
    inv_vv = np.linalg.inv(vv)
    theta[i, ] = np.dot(inv_vv, vy)

(10, 10)

In [ ]:
##omegaをサンプリング
#双線形回帰行列の入力変数を設定
theta_vec1 = theta[d_id1, ]; theta_vec2 = theta[d_id2, ]
w = np.zeros((hhpt, k*k))
for j in range(k):
    w[:, allocation_list2[:, j]] = theta_vec1[:, j][:, np.newaxis] * theta_vec2

#多変量正規分布のパラメータ
er = U - beta_mu   #モデル誤差
wy = np.dot(w.T, er)
wwv = np.dot(w.T, w) + inv_tau2
inv_wwv = np.linalg.inv(wwv)
omega_mu = np.dot(inv_wwv, wy)